In [134]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score


In [48]:
df_train = pd.read_csv("input/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("input/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]


In [49]:
BATCH_SIZE = 128
NUM_RESIDUAL_BLOCKS = 5


In [183]:
class CustomDataset(Dataset):
    
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return torch.tensor(self.x[index], dtype=torch.double), torch.tensor(self.y[index])

train_dataset = CustomDataset(X, Y)
val_dataset = CustomDataset(X_test, Y_test)

def load_data(train_dataset, val_dataset):

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    
    return train_loader, val_loader

train_loader, val_loader = load_data(train_dataset, val_dataset)


In [79]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [119]:
class CNResidual(nn.Module):
    def __init__(self, size):
        super().__init__()
        # layers
        self.c1 = nn.Conv1d(size, size, kernel_size=5, padding=2)
        self.c2 = nn.Conv1d(size, size, kernel_size=5, padding=2)
        self.pool = nn.MaxPool1d(kernel_size=5, stride=2)
        
        
    def forward(self, x):
        # call layers
        x1 = F.relu(self.c1(x))
        x1 = self.c2(x1)
        return self.pool(F.relu(x+x1))
    
class CNet(nn.Module):

    def __init__(self):
        super().__init__()
        # layers
        self.c1 = nn.Conv1d(1, 32, kernel_size=5, padding=2)
        self.cresids = []
        for i in range(NUM_RESIDUAL_BLOCKS):
            self.cresids.append(CNResidual(32))
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 5)
        
    def forward(self, x):
        # call layers
        x = self.c1(x)
        for cresid in self.cresids:
            x = cresid(x)
        x  = x.reshape(-1, 64)
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

cnetmodel = CNet()


In [121]:
criterion = nn.CrossEntropyLoss(reduction="mean")
optimizer = torch.optim.Adam(cnetmodel.parameters(), lr=1e-2)


In [202]:
def eval(model, val_loader):
    model.eval()

    x_pred = []
    x_true = []
    i = 0
    for x, y in val_loader:
        x=x.reshape(-1, 1, 187).float()
        x_hat = model(x)
        x_hat = torch.argmax(x_hat, dim=1)
        x_pred.extend(x_hat)
        x_true.extend(y.long().detach().numpy())
    f = f1_score(y_pred=x_pred, y_true=x_true, average='micro')
    acc = accuracy_score(y_pred=x_pred, y_true=x_true)
    return f, acc


In [203]:
def train(model, train_loader, val_loader, n_epochs):
    model.train()
    for epoch in range(n_epochs):
        train_loss = 0
        i=0
        for x, y in train_loader:
            optimizer.zero_grad()
            x=x.reshape(-1, 1, 187).float()
            x_hat = model(x)
            loss = criterion(x_hat, y.long())
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            if i ==0:
                break
            i += 1
        train_loss = train_loss / len(train_loader)
        print('Epoch: {} \t Training Loss: {:.6f}'.format(epoch+1, train_loss))
        f, acc = eval(model, val_loader)
        print('Epoch: %d \t Validation f: %.2f, acc: %.2f'%(epoch+1, f, acc))

In [204]:
train(cnetmodel, train_loader, val_loader, 1)


Epoch: 1 	 Training Loss: 0.001606
Epoch: 1 	 Validation f: 0.83, acc: 0.83


In [190]:
a1 = a[:10]
y = [1,2,0,0,0,3,4,0,0,0]
accuracy_score(a1, y)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and multiclass targets

In [192]:
print(a1)

[array([1.0000000e+00, 5.7952029e-15, 2.2509292e-17, 1.0872131e-16,
       2.0306382e-17], dtype=float32), array([1.0000000e+00, 9.6865616e-17, 1.9020183e-19, 1.0836428e-18,
       1.6559413e-19], dtype=float32), array([1.0000000e+00, 2.5894441e-14, 1.2798730e-16, 5.7953105e-16,
       1.1694122e-16], dtype=float32), array([1.0000000e+00, 8.2899075e-17, 1.6158854e-19, 9.1762568e-19,
       1.4013320e-19], dtype=float32), array([1.0000000e+00, 2.8153514e-16, 6.6490111e-19, 3.6565730e-18,
       5.9001656e-19], dtype=float32), array([1.0000000e+00, 3.4586754e-17, 5.7260777e-20, 3.3911024e-19,
       4.9605378e-20], dtype=float32), array([1.0000000e+00, 8.3262952e-16, 2.3425153e-18, 1.2104974e-17,
       2.0718256e-18], dtype=float32), array([1.0000000e+00, 6.3064124e-17, 1.1582625e-19, 6.7227445e-19,
       1.0016320e-19], dtype=float32), array([1.0000000e+00, 1.5271464e-15, 4.8355498e-18, 2.5008262e-17,
       4.3729238e-18], dtype=float32), array([1.0000000e+00, 1.0188522e-15, 2.996214

In [195]:
p = np.array([[0, 1], [1, 1]])
t = np.ones(2)
print(p, t)
accuracy_score(p, t)

[[0 1]
 [1 1]] [1. 1.]


ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets